In [1]:
import pandas as pd, json

In [2]:
with open ("features.json") as json_file:
    logs=json.load(json_file)

In [3]:
for line in logs:
    for direction in logs[line]:
        with open("logs/"+line+"_"+direction+".json") as log_file:
            logs[line][direction]=json.load(log_file)  

In [4]:
logs['39A']['1']

{'rows': {'start': 1834466,
  'route_filter': -14900,
  'nan_filter': -81328,
  'negative_dur': -12,
  'outliers': -10972,
  'end': 1727254},
 'scr': 0.0020584602717167557,
 'stats': {'1': {'dur_mean': 0.0, 'dur_std': 0.0},
  '2': {'dur_mean': 158.1633398348342, 'dur_std': 72.16061664018345},
  '3': {'dur_mean': 186.75755565689647, 'dur_std': 75.17811194192518},
  '4': {'dur_mean': 251.93204326026603, 'dur_std': 87.49078911311061},
  '5': {'dur_mean': 271.4036271789988, 'dur_std': 99.18156791338873},
  '6': {'dur_mean': 317.4833484835964, 'dur_std': 108.76509792468642},
  '7': {'dur_mean': 371.275787995208, 'dur_std': 119.44136073069234},
  '8': {'dur_mean': 409.4279636513837, 'dur_std': 124.82381037694539},
  '9': {'dur_mean': 471.89006403635614, 'dur_std': 136.02576299910146},
  '10': {'dur_mean': 537.4442103523775, 'dur_std': 146.14950490035613},
  '11': {'dur_mean': 571.6893917495973, 'dur_std': 153.2348574024715},
  '12': {'dur_mean': 642.0591999339471, 'dur_std': 167.560351128245

In [5]:
features={}
for line in logs:
    features[line]={}
    for direction in logs[line]:
        features[line][direction]=logs[line][direction]["features"]

In [6]:
with open("features.json","w") as json_file:
    json.dump(features,json_file)

In [7]:
acc_columns=["line","direction","n","r_2","mse","rmse","mae","b_r_2","b_mse","b_rmse","b_mae"]

In [8]:
acc_df=pd.DataFrame(columns=acc_columns)

In [9]:
for line in logs:
    for direction in logs[line]:
        m_stats=logs[line][direction]["modelling"]["XGBoost"]
        b_stats=logs[line][direction]["modelling"]["Benchmark"]
        acc_row=[
            line,
            direction,
            logs[line][direction]["rows"]["end"],
            m_stats["r_2"],
            m_stats["mse"],
            m_stats["rmse"],
            m_stats["mae"],
            b_stats["r_2"],
            b_stats["mse"],
            b_stats["rmse"],
            b_stats["mae"],
        ]
        acc_df=acc_df.append(pd.DataFrame([acc_row],columns=acc_columns),ignore_index=True)

In [10]:
acc_df["d_r_2"]=acc_df["r_2"]-acc_df["b_r_2"]

In [11]:
acc_df.sort_values("d_r_2").head(10)

,line,direction,n,r_2,mse,rmse,mae,b_r_2,b_mse,b_rmse,b_mae,d_r_2
8,16,1,1466048,0.970100,105120.930055,324.223580,225.442632,0.947896,183186.664778,428.003113,296.832592,0.022204
140,46A,2,2043187,0.963809,72515.755157,269.287495,189.445240,0.940587,119047.217607,345.032198,238.903831,0.023223
209,32X,1,18259,0.993287,20061.452119,141.638456,94.932896,0.969105,92330.517415,303.859371,207.182913,0.024182
145,123,1,956598,0.960045,58844.462352,242.578776,169.577392,0.935263,95342.745917,308.776207,207.616801,0.024782
11,14,2,1245201,0.971820,67196.673456,259.223212,177.766285,0.946785,126893.292434,356.220848,240.650425,0.025035
52,84,2,306659,0.986635,35741.805281,189.055032,127.890072,0.961131,103945.793048,322.406255,219.407657,0.025504
0,40,2,2133060,0.966010,115340.859867,339.618698,235.661072,0.940485,201958.827240,449.398295,308.809485,0.025525
5,27,1,1900447,0.971487,115084.152782,339.240553,234.301810,0.945727,219061.549875,468.040116,318.034114,0.025761
2,145,2,1780640,0.960794,78067.386595,279.405416,198.093787,0.934743,129941.120568,360.473467,252.965226,0.026051
9,16,2,1110377,0.970195,96213.019880,310.182237,211.451034,0.943745,181600.150691,426.145692,284.016582,0.026451


In [12]:
acc_df.sort_values("d_r_2",ascending=False).head(10)

,line,direction,n,r_2,mse,rmse,mae,b_r_2,b_mse,b_rmse,b_mae,d_r_2
120,161,2,4236,0.961434,9308.479807,96.480463,54.491739,0.265860,177197.256846,420.948045,318.316336,0.695574
124,40E,1,15182,0.979141,5458.656258,73.882720,43.901688,0.622818,98706.737882,314.176285,248.963909,0.356323
203,41B,1,60449,0.981139,19341.476667,139.073638,97.070992,0.681786,326317.438563,571.242014,285.824720,0.299353
121,161,1,18489,0.823011,59416.954943,243.755933,116.738016,0.525501,159293.925373,399.116431,186.155959,0.297510
212,68A,1,16940,0.962861,15823.091583,125.789871,89.853679,0.688535,132699.566234,364.279517,255.419362,0.274326
93,33A,2,90085,0.985399,11436.438624,106.941286,75.790368,0.722441,217406.841266,466.269065,338.740387,0.262959
214,236,1,15214,0.990370,3271.682818,57.198626,40.248623,0.758318,82106.848318,286.542926,205.419295,0.232051
139,41D,1,668,0.967963,23717.194282,154.003877,81.827039,0.743066,190207.137725,436.127433,296.514970,0.224896
217,31D,1,5663,0.987117,4440.644932,66.638164,47.092525,0.763621,81478.555791,285.444488,214.389124,0.223497
138,41D,2,345,0.997037,2453.848046,49.536331,34.544552,0.787900,175659.103448,419.117052,357.793103,0.209137


In [13]:
acc_df.sort_values("r_2").head(10)

,line,direction,n,r_2,mse,rmse,mae,b_r_2,b_mse,b_rmse,b_mae,d_r_2
121,161,1,18489,0.823011,59416.954943,243.755933,116.738016,0.525501,159293.925373,399.116431,186.155959,0.297510
192,53,1,73300,0.853447,34503.462147,185.751076,96.063502,0.766209,55042.155580,234.610647,126.465321,0.087238
206,25D,2,15767,0.909537,123656.990167,351.648959,247.535067,0.828641,234235.955099,483.979292,329.980467,0.080896
56,120,1,309707,0.931878,14528.014618,120.532214,89.959292,0.867336,28292.771617,168.204553,124.554277,0.064543
96,239,2,93414,0.935846,67542.470407,259.889343,169.805228,0.797756,212927.467415,461.440643,301.866661,0.138091
232,150,1,516666,0.938059,28145.160834,167.765196,123.916538,0.864019,61787.999683,248.571921,184.765590,0.074040
95,270,1,93356,0.938331,10007.057853,100.035283,61.111098,0.800904,32307.248682,179.742173,112.973264,0.137427
27,15B,1,769627,0.938340,69342.491197,263.329625,185.646564,0.872267,143647.544086,379.008633,263.834096,0.066073
193,53,2,72332,0.938641,8085.065607,89.916993,64.034127,0.821737,23489.019742,153.261279,108.262125,0.116903
19,130,2,869498,0.939838,17771.823195,133.311002,93.364403,0.899582,29663.386815,172.230621,119.360501,0.040256


In [14]:
tot_r_2=sum(acc_df["n"]*acc_df["r_2"])/sum(acc_df["n"])
tot_r_2

0.9636853153310752

In [15]:
tot_b_r_2=sum(acc_df["n"]*acc_df["b_r_2"])/sum(acc_df["n"])
tot_b_r_2

0.9172856728261803

In [16]:
acc_df["d_mae"]=acc_df["mae"]-acc_df["b_mae"]

In [17]:
acc_df.sort_values("b_mae")

,line,direction,n,r_2,mse,rmse,mae,b_r_2,b_mse,b_rmse,b_mae,d_r_2,d_mae
98,185,2,59901,0.975034,3738.724531,61.145110,39.802622,0.933426,9969.737847,99.848575,66.217014,0.041608,-26.414392
94,270,2,93169,0.958574,6215.940660,78.841237,48.671396,0.904723,14296.310308,119.567179,76.929636,0.053851,-28.258240
113,14C,1,10549,0.980239,1584.493154,39.805692,25.204112,0.848652,12135.680819,110.162066,77.274450,0.131587,-52.070338
188,59,2,109694,0.962977,4411.446973,66.418725,44.496610,0.896209,12367.238404,111.208086,78.035589,0.066768,-33.538979
189,59,1,98274,0.956689,5701.441753,75.507892,54.263700,0.890086,14469.042777,120.287334,84.781635,0.066603,-30.517935
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,41X,1,16452,0.990975,36594.352085,191.296503,130.613318,0.913393,351179.975201,592.604400,402.319475,0.077582,-271.706156
126,116,2,9586,0.990918,21991.470025,148.295212,90.265391,0.844862,375670.526909,612.919674,426.201502,0.146056,-335.936111
33,33,1,338179,0.942209,193724.338774,440.141271,285.461975,0.778147,743690.797090,862.375091,545.441575,0.164062,-259.979601
100,84X,1,9556,0.959408,113117.090046,336.328842,231.623420,0.814465,517028.676015,719.047061,553.647133,0.144943,-322.023713


In [18]:
tot_mae=sum(acc_df["n"]*acc_df["mae"])/sum(acc_df["n"])
tot_mae

169.03549449704525

In [19]:
tot_b_mae=sum(acc_df["n"]*acc_df["b_mae"])/sum(acc_df["n"])
tot_b_mae

248.31134074627863

In [20]:
tot_mae-tot_b_mae

-79.27584624923338

In [21]:
n_start=0
n_end=0
d_route=0
d_nan=0
d_neg=0
d_outlier=0
for line in logs:
    for direction in logs[line]:
        n_data=logs[line][direction]["rows"]
        n_start+=n_data["start"]
        d_route+=n_data["route_filter"]
        d_nan+=n_data["nan_filter"]
        d_neg+=n_data["negative_dur"]
        d_outlier+=n_data["outliers"]
        n_end+=n_data["end"]

In [22]:
print(f"Dropped rows due to route constraint: {-d_route} ({d_route/n_start*100:.2f}%)")
print(f"Dropped rows due to incompleteness of data (NaNs): {-d_nan} ({d_nan/n_start*100:.2f}%)")
print(f"Dropped rows due to nonsensical data (negative actual durations): {-d_neg} ({d_neg/n_start*100:.2f}%)")
print(f"Dropped rows due to statistical outlier (+/- 3 SDs of mean of route to progr_number): {-d_outlier} ({d_outlier/n_start*100:.2f}%)")
print(f"Coverage of original data: {n_end/n_start*100:.2f}%.")

Dropped rows due to route constraint: 13991713 (-11.98%)
Dropped rows due to incompleteness of data (NaNs): 6297940 (-5.39%)
Dropped rows due to nonsensical data (negative actual durations): 15695 (-0.01%)
Dropped rows due to statistical outlier (+/- 3 SDs of mean of route to progr_number): 827014 (-0.71%)
Coverage of original data: 81.91%.


In [23]:
n_start

116799394

In [24]:
n_end

95667032

In [25]:
total_models=0
for line in logs:
    for direction in logs[line]:
        total_models+=1

In [26]:
print(f"Mean of observation per model: {n_end/total_models:.1f}.")

Mean of observation per model: 379631.1.


In [28]:
xgb_r_2=[]
xgb_mae=[]
ben_r_2=[]
ben_mae=[]
for line in logs:
    for direction in logs[line]:
        results=logs[line][direction]["modelling"]
        xgb_r_2.append(results["XGBoost"]["r_2"])
        xgb_mae.append(results["XGBoost"]["mae"])
        ben_r_2.append(results["Benchmark"]["r_2"])
        ben_mae.append(results["Benchmark"]["mae"])

In [36]:
results_df=pd.DataFrame({"xgb_r_2":xgb_r_2,"xgb_mae":xgb_mae,"ben_r_w":ben_r_2,"ben_mae":ben_mae})

In [37]:
results_df

,xgb_r_2,xgb_mae,ben_r_w,ben_mae
0,0.966010,235.661072,0.940485,308.809485
1,0.968104,235.046072,0.940764,315.901510
2,0.960794,198.093787,0.934743,252.965226
3,0.953091,231.144215,0.910380,317.230532
4,0.970263,237.426801,0.936630,343.488830
...,...,...,...,...
247,0.975697,85.889040,0.931775,193.498877
248,0.981724,99.605597,0.887100,256.338752
249,0.994147,84.209891,0.950947,284.060028
250,0.994179,93.281586,0.931381,336.546937


In [38]:
results_df.describe()

,xgb_r_2,xgb_mae,ben_r_w,ben_mae
count,252.000000,252.000000,252.000000,252.000000
mean,0.968797,126.365477,0.892971,235.638523
std,0.018611,53.019961,0.067940,81.146384
min,0.823011,25.204112,0.265860,66.217014
25%,0.960278,86.977934,0.881507,181.376788
50%,0.968953,122.373117,0.909501,232.140681
75%,0.980019,162.949512,0.929141,287.574149
max,0.999517,285.461975,0.969105,582.055096
